In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import numpy as np
import pickle
import json
import glob
import os
import cloudpickle

import matplotlib.pyplot as plt
%matplotlib inline

from enterprise.signals import signal_base
from enterprise.signals import gp_signals
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter
from enterprise.signals import gp_priors

from enterprise_extensions import model_utils, blocks
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise.signals import selections

import h5py
import la_forge.core as co

In [2]:
with open('/Users/physics/nanogw/data/ppta_dr2.pkl','rb') as fin:
    psrs=pickle.load(fin)

In [3]:
for p in psrs:
    p.name=p.name+'_ppta'
for p in psrs:
    print(p.name)

J0437-4715_ppta
J0613-0200_ppta
J0711-6830_ppta
J1017-7156_ppta
J1022+1001_ppta
J1024-0719_ppta
J1045-4509_ppta
J1125-6014_ppta
J1446-4701_ppta
J1545-4550_ppta
J1600-3053_ppta
J1603-7202_ppta
J1643-1224_ppta
J1713+0747_ppta
J1730-2304_ppta
J1732-5049_ppta
J1744-1134_ppta
J1824-2452A_ppta
J1832-0836_ppta
J1857+0943_ppta
J1909-3744_ppta
J1939+2134_ppta
J2124-3358_ppta
J2129-5721_ppta
J2145-0750_ppta
J2241-5236_ppta


In [4]:
noisefile = '/Users/physics/nanogw/data/ppta_dr2_TN_noise.json'

with open(noisefile, 'r') as f:
    noisedict = json.load(f)

In [5]:
noisedictcp=noisedict.copy()
#for ky,val in noisedict.items():
#    if 'equad'in ky and not 'tnequad' in ky:
#        kynew=ky.replace('equad','tnequad')
#        noisedictcp[kynew] = noisedict[ky]

noisedictcp2=noisedictcp.copy()
for ky,val in noisedictcp.items():
    split=ky.split('_')
    split.insert(1,'ppta')
    kynew='_'.join(split)
    noisedictcp2[kynew] = noisedictcp[ky]
for ky,val in noisedictcp.items():
    if ky in noisedictcp:
        noisedictcp2.pop(ky)

In [ ]:
'''
#Run to save renamed files for ease
noise=noisedictcp2
# create a binary pickle file 
f = open("/Users/physics/nanogw/data/ppta_noise_params_renamed.pkl","wb")
# write the python object (dict) to pickle file
pickle.dump(noise,f)
# close file
f.close()
'''

In [ ]:
%%time
# (Note: It may take a few minutes to run this cell and may require at least ~4GB RAM)
# Initialize the optimal statistic object
# You can give it a list of pulsars and the noise dictionary, and it will create the pta object for you
# Alternatively, you can make the pta object yourself and give it to the OptimalStatistic object as an argument

# find the maximum time span to set GW frequency sampling
Tspan = model_utils.get_tspan(psrs)

# Here we build the signal model
# First we add the timing model
s = gp_signals.TimingModel()

# Then we add the white noise
# There are three types of white noise: EFAC, EQUAD, 
# We use different white noise parameters for every backend/receiver combination
# The white noise parameters are held constant
#inc_ecorr======FALSE because ppta
s += blocks.white_noise_block(vary=False, inc_ecorr=False, select='backend', tnequad=True)

#DMGP
dm = blocks.dm_noise_block(gp_kernel="diag", psd="powerlaw", components=100, Tspan=Tspan)

# Next comes the individual pulsar red noise
# We model the red noise as a Fourier series with 30 frequency components, 
# with a power-law PSD
s += blocks.red_noise_block(prior='log-uniform', Tspan=Tspan, components=30)

# Finally, we add the common red noise, which is modeled as a Fourier series with 5 frequency components
# The common red noise has a power-law PSD with spectral index of 4.33
s += blocks.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, 
                                   components=5, gamma_val=4.33, name='gw')
models=[]
for p in psrs:
    if p.name not in ['J0437-4715_ppta', 'J1713+0747_ppta', 'J0613-0200_ppta','J1017-7156_ppta','J1045-4509_ppta']:
        models.append((s+dm)(p))
    elif p.name=='J0437-4715_ppta':
        models.append(s(p))
#         models.append((s+dm+j0437dip)(p))
    elif p.name=='J1713+0747_ppta':
        models.append(s(p))
#         models.append((s+dm+j1713dips)(p))
    elif p.name in [ 'J0613-0200_ppta','J1017-7156_ppta','J1045-4509_ppta']:
        models.append(s(p))
# We set up the PTA object using the signal we defined above and the pulsars

import cloudpickle
with open('/Users/physics/nanogw/data/ppta_models.pkl','wb') as fout:
    cloudpickle.dump(models,fout)


"""
pta = signal_base.PTA(models)

# We need to set the white noise parameters to the values in the noise dictionary
pta.set_default_params(noisedictcp2)

ostat = opt_stat.OptimalStatistic(psrs, pta=pta, orf='hd')
ostat_dip = opt_stat.OptimalStatistic(psrs, pta=pta, orf='dipole')
ostat_mono = opt_stat.OptimalStatistic(psrs, pta=pta, orf='monopole')
"""

In [ ]:
"""
#Code to generate and save max likelihood parameters only need to run once
#To get the files generated 
c0 = co.Core(chaindir='/Users/physics/nanogw/data/chains_ppta_dr2/')
max_like=c0.get_map_dict()
# create a binary pickle file 
f = open("/Users/physics/nanogw/data/ppta_ml_params.pkl","wb")
# write the python object (dict) to pickle file
pickle.dump(max_like,f)
# close file
f.close()
"""

In [ ]:
c0 = co.Core(chaindir='/Users/physics/nanogw/data/chains_ppta_dr2/')

In [ ]:
c0.get_map_dict()

In [11]:
with open('/Users/physics/nanogw/data/ppta_models.pkl','rb') as fin:
    ppta_models = cloudpickle.load(fin)

In [12]:
ppta_models

[enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,
 enterprise.signals.signal_base.SignalCollection,


In [13]:
full_model=[]
for m,psr in zip(ppta_models,psrs):
    full_model.append(m(psr))

TypeError: fd_system_selection_0() missing 2 required positional arguments: 'sys_flags' and 'sys_flagvals'

In [ ]:
# We set up the PTA object using the signal we defined above and the pulsars
pta = signal_base.PTA([ppta_models(p) for p in psrs])

In [ ]:
# We need to set the white noise parameters to the values in the noise dictionary
pta.set_default_params(noisedictcp2)

In [ ]:
# Load up the maximum-likelihood values for the pulsars' red noise parameters and the common red process
# These values come from the results of a Bayesian search

with open("/Users/physics/nanogw/data/ppta_ml_params.pkl",'rb') as fin:
    ml_params=pickle.load(fin)

In [ ]:
ml_paramscp=ml_params.copy()
for ky,val in ml_params.items():
    split=ky.split('_')
    split.insert(1,'ppta')
    kynew='_'.join(split)
    ml_paramscp[kynew] = ml_params[ky]
for ky,val in ml_params.items():
    if ky in ml_params:
        ml_paramscp.pop(ky)
ml_paramscp['gw_log10_A']=ml_paramscp['gw_ppta_log10_A']

In [ ]:
'''
#Run to save renamed files for ease
ml=ml_paramscp
# create a binary pickle file 
f = open("/Users/physics/nanogw/data/ppta_ml_params_renamed.pkl","wb")
# write the python object (dict) to pickle file
pickle.dump(ml,f)
# close file
f.close()
'''

In [ ]:
# Compute the optimal statistic
# The optimal statistic returns five quantities:
#  - xi: an array of the angular separations between the pulsar pairs (in radians)
#  - rho: an array of the cross-correlations between the pulsar pairs
#  - sig: an array of the uncertainty in the cross-correlations
#  - OS: the value of the optimal statistic
#  - OS_sig: the uncertainty in the optimal statistic

xi, rho, sig, OS, OS_sig = ostat.compute_os(params=ml_paramscp) #HD
print(OS, OS_sig, OS/OS_sig)

_, _, _, OS_dip, OS_sig_dip = ostat_dip.compute_os(params=ml_paramscp) #Dipole
print(OS_dip, OS_sig_dip, OS_dip/OS_sig_dip)

_, _, _, OS_mono, OS_sig_mono = ostat_mono.compute_os(params=ml_paramscp) #Monopole 
print(OS_mono, OS_sig_mono, OS_mono/OS_sig_mono)

In [ ]:
# Plot the cross-correlations and compare to the Hellings-Downs curve
# Before plotting, we need to bin the cross-correlations

def weightedavg(rho, sig):
    weights, avg = 0., 0.
    for r,s in zip(rho,sig):
        weights += 1./(s*s)
        avg += r/(s*s)
        
    return avg/weights, np.sqrt(1./weights)

def bin_crosscorr(zeta, xi, rho, sig):
    
    rho_avg, sig_avg = np.zeros(len(zeta)), np.zeros(len(zeta))
    
    for i,z in enumerate(zeta[:-1]):
        myrhos, mysigs = [], []
        for x,r,s in zip(xi,rho,sig):
            if x >= z and x < (z+10.):
                myrhos.append(r)
                mysigs.append(s)
        rho_avg[i], sig_avg[i] = weightedavg(myrhos, mysigs)
        
    return rho_avg, sig_avg

# sort the cross-correlations by xi
idx = np.argsort(xi)

xi_sorted = xi[idx]
rho_sorted = rho[idx]
sig_sorted = sig[idx]

# bin the cross-correlations so that there are the same number of pairs per bin
#number of pulsar correlations you want per bin
npairs = 65

xi_mean = []
xi_err = []

rho_avg = []
sig_avg = []

i = 0

while i < len(xi_sorted):
    
    xi_mean.append(np.mean(xi_sorted[i:npairs+i]))
    xi_err.append(np.std(xi_sorted[i:npairs+i]))

    r, s = weightedavg(rho_sorted[i:npairs+i], sig_sorted[i:npairs+i])
    rho_avg.append(r)
    sig_avg.append(s)
    
    i += npairs
    
xi_mean = np.array(xi_mean)
xi_err = np.array(xi_err)

In [ ]:
def get_HD_curve(zeta):
    
    coszeta = np.cos(zeta*np.pi/180.)
    xip = (1.-coszeta) / 2.
    HD = 3.*( 1./3. + xip * ( np.log(xip) -1./6.) )
    
    return HD/2

In [ ]:
(_, caps, _) = plt.errorbar(xi_mean*180/np.pi, rho_avg, xerr=xi_err*180/np.pi, yerr=sig_avg, marker='o', ls='', 
                            color='0.1', fmt='o', capsize=4, elinewidth=1.2)

zeta = np.linspace(0.01,180,100)
HD = get_HD_curve(zeta+1)

plt.plot(zeta, OS*HD, ls='--', label='Hellings-Downs', color='C0', lw=1.5)
plt.plot(zeta, zeta*0.0+OS_mono, ls='--', label='Monopole', color='C1', lw=1.5)
plt.plot(zeta, OS_dip*np.cos(zeta*np.pi/180), ls='--', label='Dipole', color='C2', lw=1.5)

plt.xlim(0, 180);
#plt.ylim(-3.5e-29, 3.5e-29);
plt.ylabel(r'$\hat{A}^2 \Gamma_{ab}(\zeta)$')
plt.xlabel(r'$\zeta$ (deg)');

plt.legend(loc=4);

plt.tight_layout();
plt.show();

In [ ]:
def hd_orf(pos1, pos2, diag=1.):
    """Hellings & Downs spatial correlation function."""
    if np.all(np.isclose(pos1,pos2,rtol=1e-05, atol=1e-08)):
        return diag
    else:
        omc2 = (1 - np.dot(pos1, pos2)) / 2
        return 1.5 * omc2 * np.log(omc2) - 0.25 * omc2 + 0.5

In [ ]:
zeta = np.linspace(0.01,180,100)
HD = get_HD_curve(zeta+1)

plt.plot(zeta, OS*HD, ls='--', label='Hellings-Downs', color='C0', lw=1.5)
plt.plot(zeta, zeta*0.0+OS_mono, ls='--', label='Monopole', color='C1', lw=1.5)
plt.plot(zeta, OS_dip*np.cos(zeta*np.pi/180), ls='--', label='Dipole', color='C2', lw=1.5)
plt.xlim(0, 180);
plt.ylim(-1.5e-29, 1.5e-29);
plt.ylabel(r'$\hat{A}^2 \Gamma_{ab}(\zeta)$')
plt.xlabel(r'$\zeta$ (deg)');